In [69]:
import sqlite3

import ipywidgets as widgets
import numpy as np
import tensorflow as tf
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [70]:
# Schritt 1: Daten aus der SQLite-Datenbank extrahieren
def load_data_from_db(db_path):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    cursor.execute(
        'SELECT temperature, humidity, pressure, precipitation_probability FROM weather'
    )
    data = cursor.fetchall()

    conn.close()

    # Umwandlung in NumPy-Array für einfache Handhabung
    data = np.array(data)
    X = data[:, :-1]  # temperature, humidity, pressure
    y = data[:, -1]  # precipitation_probability

    # Konvertiere precipitation_probability in 0 und 1
    y = (y == 100).astype(int)

    return X, y

In [71]:
# Lade die Daten
db_path = 'labeled.sqlite'
X, y = load_data_from_db(db_path)

# Schritt 2: Daten normalisieren und vorbereiten
scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Schritt 3: Baue das neuronale Netzwerk
# model = tf.keras.Sequential(
#     [
#         tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
#         tf.keras.layers.Dropout(0.5),  # Dropout-Schicht hinzufügen
#         tf.keras.layers.Dense(64, activation='relu'),
#         tf.keras.layers.Dropout(0.5),  # Dropout-Schicht hinzufügen
#         tf.keras.layers.Dense(1, activation='sigmoid'),
#     ]
# )
model = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
        tf.keras.layers.Dropout(0.5),  # Dropout-Schicht hinzufügen
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.5),  # Dropout-Schicht hinzufügen
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid'),
    ]
)

# Schritt 4: Kompiliere das Modell
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
# model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Schritt 5: Trainiere das Modell mit Early Stopping
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=5, restore_best_weights=True
)
history = model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping],
)

# Schritt 6: Evaluiere das Modell
loss, accuracy = model.evaluate(X_test, y_test)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)

Epoch 1/100
9/9 [==============================] - 1s 22ms/step - loss: 0.6455 - accuracy: 0.6885 - val_loss: 0.6101 - val_accuracy: 0.8636
Epoch 2/100
9/9 [==============================] - 0s 7ms/step - loss: 0.5575 - accuracy: 0.8385 - val_loss: 0.5211 - val_accuracy: 0.8636
Epoch 3/100
9/9 [==============================] - 0s 5ms/step - loss: 0.4939 - accuracy: 0.8385 - val_loss: 0.4361 - val_accuracy: 0.8636
Epoch 4/100
9/9 [==============================] - 0s 5ms/step - loss: 0.4262 - accuracy: 0.8385 - val_loss: 0.3778 - val_accuracy: 0.8636
Epoch 5/100
9/9 [==============================] - 0s 5ms/step - loss: 0.4183 - accuracy: 0.8385 - val_loss: 0.3551 - val_accuracy: 0.8636
Epoch 6/100
9/9 [==============================] - 0s 5ms/step - loss: 0.4203 - accuracy: 0.8385 - val_loss: 0.3441 - val_accuracy: 0.8636
Epoch 7/100
9/9 [==============================] - 0s 5ms/step - loss: 0.3806 - accuracy: 0.8385 - val_loss: 0.3385 - val_accuracy: 0.8636
Epoch 8/100
9/9 [=========

In [72]:
def predict_rain(temperature, humidity, pressure):
    # Vorhersage machen
    input_data = scaler.transform([[temperature, humidity, pressure]])
    prediction = model.predict(input_data)
    rain_probability = prediction[0][0]
    print(f'Predicted Rain Probability: {rain_probability:.2f}')

In [73]:
# # Interaktive Widgets erstellen
# temperature_slider = widgets.FloatSlider(
#     min=-30, max=50, step=0.1, description='Temperature:'
# )
# humidity_slider = widgets.FloatSlider(min=0, max=100, step=0.1, description='Humidity:')
# pressure_slider = widgets.FloatSlider(
#     min=900, max=1100, step=0.01, description='Pressure:'
# )
# Interaktive Textfelder erstellen
temperature_input = widgets.FloatText(value=0.0, description='Temperature:')
humidity_input = widgets.FloatText(value=0.0, description='Humidity:')
pressure_input = widgets.FloatText(value=0.0, description='Pressure:')

In [74]:
# Button erstellen, um Vorhersage auszulösen
predict_button = widgets.Button(description='Predict')
output = widgets.Output()


# def on_predict_button_clicked(b):
#     with output:
#         output.clear_output()
#         predict_rain(
#             temperature_slider.value, humidity_slider.value, pressure_slider.value
#         )
def on_predict_button_clicked(b):
    with output:
        output.clear_output()
        predict_rain(
            temperature_input.value, humidity_input.value, pressure_input.value
        )


predict_button.on_click(on_predict_button_clicked)

In [75]:
# Widgets anzeigen
# display(temperature_slider, humidity_slider, pressure_slider, predict_button, output)
display(temperature_input, humidity_input, pressure_input, predict_button, output)

FloatText(value=0.0, description='Temperature:')

FloatText(value=0.0, description='Humidity:')

FloatText(value=0.0, description='Pressure:')

Button(description='Predict', style=ButtonStyle())

Output()